In [1]:
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [16]:
#Import 
preds = pd.read_csv('../data/tables/FlorenceFourthLabels20180917a.csv')
#raw images
aerial_folder = '../data/pics/raw/Florence_20180917a_jpgs/'
#save path
ALPATH = '../data/pics/modeltoLabeler/roundFour/FlorenceWash4/'
#new csv to save
newpredcsv = '../data/tables/FlorenceFourthWithWashLabels20180917a.csv'


# #import
# preds = pd.read_csv('../data/tables/MichaelFourthLabels20181011a.csv')
# #raw images:
# aerial_folder = '../data/pics/raw/Michael_20181011a_jpgs/'
# #save path
# ALPATH = '../data/pics/modeltoLabeler/roundFour/MichaelWash4/'
# # new csv
# newpredcsv = '../data/tables/MichaelFourthWithWashLabels20181011a.csv'


# #Import
# preds = pd.read_csv('../data/tables/IsaiasFourthLabels20200804a.csv')
# #raw images:
# aerial_folder = '../data/pics/raw/Isaias_20200804a_jpgs/'
# #save path
# ALPATH = '../data/pics/modeltoLabeler/roundFour/IsaiasWash4/'
# #new csv
# newpredcsv = '../data/tables/IsaiasFourthWithWashLabels20200804a.csv'

In [17]:
preds.head()

,Unnamed: 0,file,storm_id,archive,image,size,date,ul_lat,ur_lat,geom_checksum,...,ll_lon,prediction,pred_certainty,labeled,washover,prediction2,pred_certainty2,washImages,prediction3,pred_certainty3
0,1759,20180917a_jpgs/jpgs/P26058502.jpg,florence,20180917a_jpgs,P26058502.jpg,6174041,2018/09/17,34.928300,34.931574,827ccdd65dc6e468901b9074188aa67b,...,-76.220528,0.499669,0.000331,2.0,NaN,0.971279,0.471279,0.221279,0.993054,0.493054
1,2977,20180917a_jpgs/jpgs/P26059434.jpg,florence,20180917a_jpgs,P26059434.jpg,5350431,2018/09/17,35.235277,35.230687,ee6643d42ab5e4b7613167a6ed657832,...,-75.553097,0.499333,0.000667,2.0,NaN,0.756587,0.256587,0.006587,0.670039,0.170039
2,1814,20180917a_jpgs/jpgs/P26057539.jpg,florence,20180917a_jpgs,P26057539.jpg,5689109,2018/09/17,34.672949,34.670919,d4986ea1ecee8ca54ab5e353af63baae,...,-76.608181,0.501626,0.001626,2.0,NaN,0.807254,0.307254,0.057254,0.998793,0.498793
3,2606,20180917a_jpgs/jpgs/P26059328.jpg,florence,20180917a_jpgs,P26059328.jpg,5959711,2018/09/17,35.231828,35.232004,31fc396ed145b440f6028ceb9797ce7b,...,-75.638167,0.497229,0.002771,2.0,NaN,0.709799,0.209799,0.040201,0.867434,0.367434
4,1752,20180917a_jpgs/jpgs/P26057234.jpg,florence,20180917a_jpgs,P26057234.jpg,6590240,2018/09/17,34.696049,34.695104,23f7e3c770a59aff24b40fce3334365d,...,-76.830926,0.503741,0.003741,2.0,NaN,0.795989,0.295989,0.045989,0.391272,0.108728


In [18]:
#should be 500 (2*100 + 100*3)... 882 for florence
np.nansum(preds.labeled)

1582.0

In [19]:
#make new column certainty of prediction, from 0 to 0.5... 0 is smallest certainty
preds["washImages3"] = abs(preds["prediction3"].astype(float) - 0.95)

#select top 100 each that have not been labeled
uncertain_preds = preds[preds.labeled.isna()].nsmallest(100, 'washImages3')

maxval = max(uncertain_preds['washImages3'])

#add labeled = 3 for all the images that will be labeled
preds.loc[(preds['washImages3'] <= maxval) & (preds['labeled'].isna()), 'labeled'] = 4

#preds.head()
uncertain_preds.head()

,Unnamed: 0,file,storm_id,archive,image,size,date,ul_lat,ur_lat,geom_checksum,...,prediction,pred_certainty,labeled,washover,prediction2,pred_certainty2,washImages,prediction3,pred_certainty3,washImages3
1020,3167,20180917a_jpgs/jpgs/P26059283.jpg,florence,20180917a_jpgs,P26059283.jpg,6315865,2018/09/17,35.218790,35.220594,3a9b7684630f09699ceacbeda22b7e5a,...,0.953639,0.453639,NaN,NaN,0.840299,0.340299,0.090299,0.950102,0.450102,0.000102
536,398,20180917a_jpgs/jpgs/P26058981.jpg,florence,20180917a_jpgs,P26058981.jpg,6398023,2018/09/17,35.134601,35.136884,c1751beb7f865b0de04764cd28795a8d,...,0.827916,0.327916,NaN,NaN,0.937756,0.437756,0.187756,0.950393,0.450393,0.000393
1377,894,20180917a_jpgs/jpgs/P26056516.jpg,florence,20180917a_jpgs,P26056516.jpg,6111445,2018/09/17,34.511887,34.518148,f130bcf7931c3f12673f92dab66000de,...,0.992150,0.492150,NaN,NaN,0.819904,0.319904,0.069904,0.949247,0.449247,0.000753
653,2813,20180917a_jpgs/jpgs/P26059138.jpg,florence,20180917a_jpgs,P26059138.jpg,5971083,2018/09/17,35.182569,35.183988,d75544b69a2bb52e6d8e87e840e3fdc6,...,0.141104,0.358896,NaN,NaN,0.796997,0.296997,0.046997,0.949046,0.449046,0.000954
640,3267,20180917a_jpgs/jpgs/P26060206.jpg,florence,20180917a_jpgs,P26060206.jpg,6135122,2018/09/17,35.588856,35.599774,31fee2059b1584bc9f8373f89945a414,...,0.856505,0.356505,NaN,NaN,0.841042,0.341042,0.091042,0.948738,0.448738,0.001262


In [20]:
#should be 500 (200*3 + 100*2).... 882 + 300 = 1182 for florence
np.nansum(preds.labeled)

1982.0

In [21]:
#save the data
preds.to_csv(newpredcsv, index=False)

In [22]:
#add the path
uncertain_preds['Image_Path'] = aerial_folder + uncertain_preds['image'].astype(str)

#pull out the list from the df
Labeled_list= uncertain_preds['Image_Path'].tolist()

#move those on the list
for f in Labeled_list:
    shutil.copy(f, ALPATH)

In [ ]:
# 